# Invoke Bedrock model for SQL Query Generation

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

## Introduction

In this notebook we show you how to use a LLM to generate SQL Query to analyze Sales data.

We will use Bedrock's Claude V2 model using the Boto3 API. 

The prompt used in this example is called a zero-shot prompt because we are not providing any examples of text other than the prompt.

**Note:** *This notebook can be run within or outside of AWS environment.*

#### Context
To demonstrate the SQL code generation capability of Amazon Bedrock, we will explore the use of Boto3 client to communicate with Amazon Bedrock API. We will demonstrate different configurations available as well as how simple input can lead to desired outputs.

#### Pattern
We will simply provide the Amazon Bedrock API with an input consisting of a task, an instruction and an input for the model under the hood to generate an output without providing any additional example. The purpose here is to demonstrate how the powerful LLMs easily understand the task at hand and generate compelling outputs.

![](./images/sql-query-generation.png)

#### Use case
Let's take the use case to generate SQL queries to analyze sales data, focusing on trends, top products and average sales.

#### Persona
Maya is a business analyst, at AnyCompany primarily focusing on sales and inventory data. She is transitioning from Speadsheet analysis to data-driven analysis and want to use SQL to fetch specific data points effectively. She wants to use LLMs to generate SQL queries for her analysis. 

#### Implementation
To fulfill this use case, in this notebook we will show how to generate SQL queries. We will use the Anthropic Claude v2 model using the Amazon Bedrock API with Boto3 client. 

## Setup

⚠️ ⚠️ ⚠️ Before running this notebook, ensure you've run the [Bedrock boto3 setup notebook](../00_Intro/bedrock_boto3_setup.ipynb#Prerequisites) notebook. ⚠️ ⚠️ ⚠️


In [2]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

Create new client
  Using region: us-east-1
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


## Generate SQL Query

Following on the use case explained above, let's prepare an input for  the Amazon Bedrock service to generate SQL query.

In [3]:
# create the prompt to generate SQL query
prompt_data = """

Human: AnyCompany has a database with a table named sales_data containing sales records. The table has following columns:
- date (YYYY-MM-DD)
- product_id
- price
- units_sold

Can you generate SQL queries for the below: 
- Identify the top 5 best selling products by total sales for the year 2023
- Calculate the monthly average sales for the year 2023

Assistant:
"""


Let's start by using the Anthorpic Claude v2 model. 

In [4]:
# Claude - Body Syntex
body = json.dumps({
                    "prompt": prompt_data,
                    "max_tokens_to_sample":4096,
                    "temperature":0.5,
                    "top_k":250,
                    "top_p":0.5,
                    "stop_sequences": ["\n\nHuman:"]
                  }) 

#### Invoke the Bedrock's Claude Large Large language model

First, we explore how the model generates an output based on the prompt created earlier.

##### Complete Output Generation

In [5]:
modelId = 'anthropic.claude-v2' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'

response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

print_ww(response_body.get('completion'))

 Here are the SQL queries to answer the questions:

1. Identify the top 5 best selling products by total sales for the year 2023:

```sql
SELECT product_id, SUM(price * units_sold) AS total_sales
FROM sales_data
WHERE date BETWEEN '2023-01-01' AND '2023-12-31'
GROUP BY product_id
ORDER BY total_sales DESC
LIMIT 5;
```

2. Calculate the monthly average sales for the year 2023:

```sql
SELECT
  DATE_FORMAT(date, '%Y-%m') AS month,
  AVG(price * units_sold) AS avg_monthly_sales
FROM sales_data
WHERE date BETWEEN '2023-01-01' AND '2023-12-31'
GROUP BY month
ORDER BY month;
```

The first query groups the sales data by product_id, sums the total sales for each product, filters
for 2023 data only, orders by the total sales in descending order and limits to the top 5 results.

The second query extracts the month from the date, calculates the average monthly sales by
aggregating on the month and ordering the results chronologically.


### Advanced Example
#### Understanding Hospital's Patient Management System through SQL

In [6]:
# create the prompt
prompt_sql_data = """

Human: You're provided with a database schema representing any hospital's patient management system.
The system holds records about patients, their prescriptions, doctors, and the medications prescribed.

Here's the schema:

```sql
CREATE TABLE Patients (
    PatientID int,
    FirstName varchar(50),
    LastName varchar(50),
    DateOfBirth datetime,
    Gender varchar(10),
    PRIMARY KEY (PatientID)
);

CREATE TABLE Doctors (
    DoctorID int,
    FirstName varchar(50),
    LastName varchar(50),
    Specialization varchar(50),
    PRIMARY KEY (DoctorID)
);

CREATE TABLE Prescriptions (
    PrescriptionID int,
    PatientID int,
    DoctorID int,
    DateIssued datetime,
    PRIMARY KEY (PrescriptionID)
);

CREATE TABLE Medications (
    MedicationID int,
    MedicationName varchar(50),
    Dosage varchar(50),
    PRIMARY KEY (MedicationID)
);

CREATE TABLE PrescriptionDetails (
    PrescriptionDetailID int,
    PrescriptionID int,
    MedicationID int,
    Quantity int,
    PRIMARY KEY (PrescriptionDetailID)
);
```

Write a SQL query that fetches all the patients who were prescribed more than 5 different medications on 2023-04-01.

Assistant:
"""


In [7]:
# Claude - Body Syntex
body = json.dumps({
                    "prompt": prompt_sql_data,
                    "max_tokens_to_sample":4096,
                    "temperature":0.5,
                    "top_k":250,
                    "top_p":0.5,
                    "stop_sequences": ["\n\nHuman:"]
                  }) 

In [8]:
modelId = 'anthropic.claude-v2' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'

response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

print_ww(response_body.get('completion'))

 Here is a SQL query to fetch patients who were prescribed more than 5 different medications on
2023-04-01:

```sql
SELECT p.FirstName, p.LastName
FROM Patients p
JOIN Prescriptions pre ON p.PatientID = pre.PatientID
JOIN PrescriptionDetails pd ON pre.PrescriptionID = pd.PrescriptionID
WHERE pre.DateIssued = '2023-04-01'
GROUP BY p.PatientID
HAVING COUNT(DISTINCT pd.MedicationID) > 5;
```

The key steps are:

1. Join the Patients, Prescriptions and PrescriptionDetails tables to connect patients with their
prescriptions and medication details.

2. Filter to only prescriptions issued on 2023-04-01.

3. Group by PatientID and count the distinct MedicationIDs per patient.

4. Use HAVING to only keep patients with more than 5 distinct medications.

5. Select the patient FirstName and LastName in the final output.


In [ ]:
# compare with query generated with ChatGPT

SELECT P.PatientID, P.FirstName, P.LastName
FROM Patients P
JOIN (
    SELECT PD.PatientID, COUNT(DISTINCT PD.MedicationID) AS MedicationCount
    FROM (
        SELECT PR.PatientID, PD.MedicationID
        FROM Prescriptions PR
        JOIN PrescriptionDetails PD ON PR.PrescriptionID = PD.PrescriptionID
        WHERE PR.DateIssued = '2023-04-01'
    ) PD
    GROUP BY PD.PatientID
    HAVING COUNT(DISTINCT PD.MedicationID) > 5
) Subquery
ON P.PatientID = Subquery.PatientID;


## Conclusion
You have now experimented with using `boto3` SDK which provides a vanilla exposure to Amazon Bedrock API. Using this API you have seen the use cases of generate SQL queries to analyze sales data.

### Take aways
- Adapt this notebook to experiment with different models available through Amazon Bedrock such as Anthropic Claude and AI21 Labs Jurassic models.
- Apply different prompt engineering principles to get better outputs.

## Thank You